In [1]:
import torch
import numpy as np
from hyperfast import HyperFastClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
data = pd.read_csv('./val.csv')
print(len(data))

28462


In [30]:
data = data.drop(columns=['ID'])
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,V29,V30,Class
0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,-0.366846,...,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,-0.255991,-0.994878,0
1,0.962496,0.328461,-0.171479,2.109204,1.129566,1.696038,0.107712,0.521502,-1.191311,0.724396,...,0.402492,-0.048508,-1.371866,0.390814,0.199964,0.016371,-0.014605,0.168937,-0.994784,0
2,1.145524,0.575068,0.194008,2.598192,-0.092210,-1.044430,0.531588,-0.241888,-0.896287,0.757952,...,-0.119703,-0.076510,0.691320,0.633984,0.048741,-0.053192,0.016251,0.169496,-0.994502,0
3,0.927060,-0.323684,0.387585,0.544474,0.246787,1.650358,-0.427576,0.615371,0.226278,-0.225495,...,0.079359,0.096632,-0.992569,0.085096,0.377447,0.036096,-0.005960,0.331307,-0.994467,0
4,-3.005237,2.600138,1.483691,-2.418473,0.306326,-0.824575,2.065426,-1.829347,4.009259,6.051521,...,-0.181268,-0.163747,0.515821,0.136318,0.460054,-0.251259,-1.105751,-0.287012,-0.994373,0


In [31]:
cnt = data[data['Class'] == 1].shape[0]

In [32]:
class1 = data[data['Class']==1]
other = data[data['Class'] != 1]
print(len(class1), len(other))

30 28432


In [33]:
class_1_train, class_1_test = train_test_split(class1, test_size=0.1, random_state=42)

# 나머지 데이터를 train과 test로 나눕니다.
other_train, other_test = train_test_split(other, test_size=0.1, random_state=42)

# 각각의 train 데이터를 합칩니다.
train_data = pd.concat([class_1_train, other_train])

# 각각의 test 데이터를 합칩니다.
test_data = pd.concat([class_1_test, other_test])
print(len(train_data), len(test_data))

25615 2847


In [34]:
x_train = train_data.drop(columns=['Class']).to_numpy()
y_train = train_data['Class'].to_numpy()

# test_data를 x_test와 y_test로 나눕니다.
x_test = test_data.drop(columns=['Class']).to_numpy()
y_test = test_data['Class'].to_numpy()

np.random.seed(42)
train_indices = np.random.permutation(x_train.shape[0])
x_train_np = x_train[train_indices]
y_train_np = y_train[train_indices]

# test 데이터를 섞습니다.
test_indices = np.random.permutation(x_test.shape[0])
x_test_np = x_test[test_indices]
y_test_np = y_test[test_indices]

In [35]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [36]:
model = HyperFastClassifier(device=device)

# Generate a target network and make predictions
model.fit(x_train_np, y_train_np)
predictions = model.predict(x_test_np)

accuracy = accuracy_score(y_test_np, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

class_1_indices = np.where(y_test_np == 1)[0]
class_1_accuracy = accuracy_score(y_test_np[class_1_indices], predictions[class_1_indices])
print(f"Class 1 Accuracy: {class_1_accuracy * 100:.2f}%")

# class=1이 아닌 경우의 정확도를 계산합니다.
class_0_indices = np.where(y_test_np != 1)[0]
class_0_accuracy = accuracy_score(y_test_np[class_0_indices], predictions[class_0_indices])
print(f"Class 0 Accuracy: {class_0_accuracy * 100:.2f}%")

/home/a2019312297/miniconda3/envs/hyperfast/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


RuntimeError: stack expects each tensor to be equal size, but got [2847, 2] at entry 0 and [2847, 1] at entry 4